## 股票策略分析

### 作业要求：

##### 所有回测场景下，设定初始资金100万，交易手续费万分之二

##### Task1（15分）：补全给出的基于元编程的回测系统的元类代码（metabase.py文件中）——只有补全了才能正常使用回测系统


- 补全`findbases`函数（5分）：实现寻找类的所有基（父）类，包括间接的基类——递归实现

- 出题的思路：让同学了解python中继承关系的遍历方式与实际继承情况

- 补全`findowner`函数（5分）：在堆栈帧中查找拥有特定类实例的对象

- 出题的思路：让同学了解python中实例化关系的储存模式——栈的理解

- 补全`AutoInfoClass**类中的``_derive`函数（5分）：实现了类型对象间信息的传递

- 出题的思路：让同学了解Python中对象性质的解构方式

- 额外任务（有助于理解回测架构）：描述数据进入回测系统后在各组件间的传递情况（最多加5分，加到满分为止）

##### Task2（25分）：在给定的股票数据下，将以下两种策略结合起来实现一个综合性的策略，用量化回测系统进行回测，呈现结果

- 这部分只要能设计出策略实现盈利就可以得到15分，只要能够使用回测系统呈现基本指标：如年化回报、波动率、夏普比例等就可以得到5分，最后能够绘制出图像就可以得到最后5分

- （如果使用pyfolio库，因为其中有若干bug，需要手动修复）https://blog.csdn.net/Yangxh2004/article/details/121451734

- 股票策略（推荐参考《151 trading strategies》）

- 动量策略：是分析过去一段时间中，多个股票上的收益率，做多收益率最高的一组；做空收益率最低的一组，首先，计算过去60个交易日股票的累计收益率，根据累计收益率的高低，把股票分为10组，做多收益率高的一组，做空收益率低的一组。资金在各个股票之间等权重分配。
  
- 均线策略：我们使用全市场的A股日数据进行测试，只做多头。首先，在当前交易日，分析一下，有多少个股票的均线已经存在，然后把资金平分成多少份；当前一个收盘价小于均线，这个收盘价大于均线的时候，做多这支股票，根据资金，计算出应该买的股票数目；当前一个收盘价大于均线，当前的收盘价小于均线的时候，平仓这支股票。

##### Task3（30分）：股票数据的预处理与股票的聚类：请参考以下链接中的基本框架进行优化  
https://github.com/areed1192/sigma_coding_youtube/blob/master/python/python-data-science/machine-learning/k-means/Clustering%20Stocks%20-%20KMeans.ipynb

- 读取数据、清洗数据（5分）：从stock中读取股票的行情和基本面数据、处理各支股票中的缺失值和异常值
- 特征提取：使用以下几类方法对时间序列数据特征提取

- 基本统计特征：均值、中位数、方差、极值、峰度和偏度（5分）
- 时域特征：差分、移动均值、滞后特征（5分）https://zhuanlan.zhihu.com/p/398752292
- 频域特征：傅里叶变换或小波变换提取频域信息（5分）https://www.zhihu.com/question/24021704/answer/2245867156
- 深度学习特征提取：利用预训练的模型进行特征提取（如基于BERT等预训练模型）——选做（加分最多5分，加到满分为止）——https://zhuanlan.zhihu.com/p/686888794

- 特征融合：归一化数据，需要将不同的特征向量组合成一个整体特征向量，并使用PCA方法降维（可设置设置累积方差贡献率阈值确定留下多少主成分）（5分）
- 股票聚类：使用K-means进行股票聚类。肘部法确定最佳簇数量——输出每个簇的代表股票、簇中股票的数量、年化回报分布等，也可对结果进行可视化展示。（5分）


##### Task4（30分）：基于以上提取出的股票特征、聚类信息和使用过的传统策略，设计股票策略并评估结果

- 按照时间先后划分测试和训练集，测试集在前训练集在后，训练集:测试集=7:3
- 设计策略，将聚类信息与均线、动量策略结合，指导股票的分组处理（10分）
- 在已知数据集对策略进行训练和调优
- 将策略应用于未知数据集，并评估绩效（20分）
  ——最低要求是不比Task2中的策略的绩效差，具体得分会按照选择此作业的同学的分布情况给出，同学的绩效分布会进行在群里公开

- 传统金融指标：年化回报率、年化波动率
- 投资效用：计算效用函数，值越高越好

对于第i个有交易的天数，定义：$p_{i}=\sum_{j}weight_{ij}*resp_{ij} , \ t=\frac{\sum p_{i}}{\sqrt{ \sum p_{i}^{2} }}\sqrt{ \frac{T}{\lvert i \rvert } }$

其中，$weight_{ij}$是该天编号为j的一笔交易的量占当天总交易量的比重，$resp_{ij}$是这笔交易的回报率，T是测试集的总天数，$\lvert i \rvert$是有交易的总天数

则效用函数定义:$u=min(max(t,0),6)\sum p_{i}$

使用该指标进行评估的参考比赛：https://www.kaggle.com/competitions/jane-street-market-prediction


In [7]:
# Task 1 补全./backtesting/metabase.py中的三个函数b

In [8]:
# Task2 在给定的股票数据下，将以下两种策略结合起来实现一个综合性的策略，用量化回测系统进行回测，呈现结果
import backtesting as bt
import datetime
import pandas as pd
import numpy as np
import os
import pyfolio as pf
import warnings

warnings.filterwarnings("ignore")
cerebro = bt.Cerebro()



- 动量策略：是分析过去一段时间中，多个股票上的收益率，做多收益率最高的一组；做空收益率最低的一组，首先，计算过去60个交易日股票的累计收益率，根据累计收益率的高低，把股票分为10组，做多收益率高的一组，做空收益率低的一组。资金在各个股票之间等权重分配。
  
- 均线策略：我们使用全市场的A股日数据进行测试，只做多头。首先，在当前交易日，分析一下，有多少个股票的均线已经存在，然后把资金平分成多少份；当前一个收盘价小于均线，这个收盘价大于均线的时候，做多这支股票，根据资金，计算出应该买的股票数目；当前一个收盘价大于均线，当前的收盘价小于均线的时候，平仓这支股票。

In [9]:
params=dict(
    fromdate = datetime.datetime(2020, 1, 2),
    todate = datetime.datetime(2022, 12, 30),
    timeframe = bt.TimeFrame.Days,
    compression = 1,
    datetime = 0,
    open = 1,
    high = 2,
    low =3,
    close =4,
    volume =5,
    openinterest=-1)
    
# 读取数据
stock_hfg_df = pd.read_feather('./data/stk_daily.feather')
grouped_data = stock_hfg_df.groupby('stk_id')
for stk_id, data in grouped_data:
    data = data.drop(columns = ['stk_id'])
    data['date'] = pd.to_datetime(data['date'])
    data = data.set_index('date')
    feed = bt.feeds.PandasData(dataname = data,**params)
    cerebro.adddata(feed, name = stk_id)

In [10]:
cerebro.broker.setcommission(commission=0.0002,stocklike=True)
cerebro.broker.setcash(1000000.0)

In [11]:
class test_divdend_rate_strategy(bt.Strategy):
   
    params = (('window',200),)

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('{}, {}'.format(dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.bar_num=0
        #要求./stock/股票历史股息率数据.csv的SECID是file_list的前20个
        df = pd.read_csv("./stock/股票历史股息率数据.csv",index_col=0)
        self.stock_divdend_info = df[df['secID'].isin(file_llist)]
        # self.stock_divdend_info = pd.read_csv("./stock/股票历史股息率数据.csv",index_col=0)
        self.buy_list=[]
        self.value_list = []
        self.trade_list = []
        self.order_list = []
       
    def prenext(self):
        
        self.next()
        
        
    def next(self):
        # 假设有100万资金，每次成份股调整，每个股票使用1万元
        self.bar_num+=1
        self.log(self.bar_num)
        # 需要调仓的时候
        pre_current_date = self.datas[0].datetime.date(-1).strftime("%Y-%m-%d")
        current_date = self.datas[0].datetime.date(0).strftime("%Y-%m-%d")
        total_value = self.broker.get_value()
        self.value_list.append([current_date,total_value])
        # 如果是8月的第一个交易日
        if current_date[5:7]=='08' and pre_current_date[5:7]!='08':
            # 获取当前股息率前10的股票
            divdend_info  = self.stock_divdend_info[self.stock_divdend_info['tradeDate']==current_date]
            divdend_info = divdend_info.sort_values("divRate",ascending=False)
            divdend_info = divdend_info.drop_duplicates("secID")
            divdend_stock_list= [i.split('.')[0] for i in list(divdend_info['secID'])]
            if len(divdend_stock_list)>10:
                stock_list= divdend_stock_list[:10]
            else:
                stock_list= divdend_stock_list
            self.log(stock_list)
            # 平掉原来的仓位
            for stock in self.buy_list:
                data = self.getdatabyname(stock)
                if self.getposition(data).size>0:
                    self.close(data)
            # 取消所有未成交的订单
            for order in self.order_list:
                self.cancel(order)
                # self.log(order)
            self.buy_list = stock_list
            
            value =0.90*self.broker.getvalue()/len(stock_list)
            # 开新的仓位，按照90%的比例开
            for stock in stock_list:
                data = self.getdatabyname(stock)
                # 没有把手数设定为100的倍数
                lots = value/data.close[0]
                order = self.buy(data,size = lots)
                self.log(f"symbol:{data._name},price:{data.close[0]}")
                self.order_list.append(order)
            
    
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order被提交和接受
            return
        if order.status == order.Rejected:
            self.log(f"order is rejected : order_ref:{order.ref}  order_info:{order.info}")
        if order.status == order.Margin:
            self.log(f"order need more margin : order_ref:{order.ref}  order_info:{order.info}")
        if order.status == order.Cancelled:
            self.log(f"order is concelled : order_ref:{order.ref}  order_info:{order.info}")
        if order.status == order.Partial:
            self.log(f"order is partial : order_ref:{order.ref}  order_info:{order.info}")
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status == order.Completed:
            if order.isbuy():
                self.log("buy result : buy_price : {} , buy_cost : {} , commission : {}".format(
                            order.executed.price,order.executed.value,order.executed.comm))
                
            else:  # Sell
                self.log("sell result : sell_price : {} , sell_cost : {} , commission : {}".format(
                            order.executed.price,order.executed.value,order.executed.comm))
    
    def notify_trade(self, trade):
        # 一个trade结束的时候输出信息
        if trade.isclosed:
            self.log('closed symbol is : {} , total_profit : {} , net_profit : {}' .format(
                            trade.getdataname(),trade.pnl, trade.pnlcomm))
            self.trade_list.append([self.datas[0].datetime.date(0),trade.getdataname(),trade.pnl,trade.pnlcomm])
            
        if trade.isopen:
            self.log('open symbol is : {} , price : {} ' .format(
                            trade.getdataname(),trade.price))
    def stop(self):
        pass
        value_df = pd.DataFrame(self.value_list)
        value_df.columns=['datetime','value']
        value_df.to_csv("股息率value结果_狗股0.csv")
        
        trade_df = pd.DataFrame(self.trade_list)
        # trade_df.columns =['datetime','name','pnl','net_pnl']
        trade_df.to_csv("股息率-trade结果_狗股0.csv")
            

In [12]:
cerebro.addstrategy(test_divdend_rate_strategy)
cerebro.addanalyzer(bt.analyzers.TotalValue, _name='_TotalValue')
cerebro.addanalyzer(bt.analyzers.PyFolio)
# 运行回测
results = cerebro.run()
# 打印相关信息

pyfoliozer = results[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    # gross_lev=gross_lev,
    live_start_date='2019-01-01',
    )


AttributeError: 'numpy.float64' object has no attribute 'to_pydatetime'

In [ ]:
# Task3 股票数据的预处理与股票的聚类

In [ ]:
# Task4 基于以上提取出的股票特征、聚类信息和使用过的传统策略，设计股票策略并评估结果